# **Reward Model Safety Evaluation**

**Author: Jin Thau - DSA4213 Group 18**

This notebook evaluates the trained Reward Model with a focus on the **key safety metric**:
**Rate of high-confidence errors in high-risk medical scenarios**

Specifically measures:
1. High-confidence error rates stratified by risk level
2. Calibration quality per risk category
3. Confidence-risk heatmaps
4. Safety-critical failure analysis

---
## **Table of Contents**
1. [Setup & Imports](#setup)
2. [Load Model & Data](#load)
3. [Safety Metrics Computation](#metrics)
4. [Visualizations](#viz)
5. [Baseline Comparison](#baseline)

---
# 1. Setup & Imports <a id='setup'></a>

In [ ]:
from collections import defaultdict
from pathlib import Path
from typing import Dict, List, Tuple
import json, numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import torch
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [ ]:
# ╭────────────────────────────── Configuration ───────────────────────────────╮
# Directories
CUR_DIR = Path.cwd()
SRC_DIR = CUR_DIR.parent
DATA_DIR = SRC_DIR.parent / "data" / "processed"
MODEL_DIR = CUR_DIR / "save_model"  # Your saved model
EVAL_DIR = CUR_DIR / "evaluation"
VIZ_DIR = EVAL_DIR / "visualizations"

# Create directories
for p in (EVAL_DIR, VIZ_DIR):
    p.mkdir(parents=True, exist_ok=True)

# Data paths
VAL_PATH = DATA_DIR / "preference_pairs_val.json"

# Evaluation parameters
MAX_LEN = 1024
BATCH_SIZE = 4
CONFIDENCE_THRESHOLD = 0.8  # Define "high confidence"
RISK_LEVELS = ["Low Risk", "Medium Risk", "High Risk"]

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 2. Load Model & Data <a id='load'></a>

In [ ]:
# Load trained model and tokenizer
print(f"Loading model from {MODEL_DIR}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)
model = model.to(device)
model.eval()
print("✓ Model loaded successfully")

In [ ]:
# Load validation/test data
def load_json(path: Path) -> List[Dict]:
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

val_pairs = load_json(VAL_PATH)
print(f"Loaded {len(val_pairs)} validation pairs")

eval_pairs = val_pairs  # Use validation set for now

In [ ]:
# Dataset class (same as training)
def build_input(q, a):
    return f"Question:\n{q}\n\nAnswer:\n{a}"

class PrefPairDataset(Dataset):
    def __init__(self, items):
        self.items = items
    def __len__(self):
        return len(self.items)
    def __getitem__(self, i):
        ex = self.items[i]
        x_pos = build_input(ex["question_text"], ex["preferred_answer"]["answer_text"])
        x_neg = build_input(ex["question_text"], ex["rejected_answer"]["answer_text"])
        pos = tokenizer(x_pos, truncation=True, max_length=MAX_LEN, return_tensors="pt")
        neg = tokenizer(x_neg, truncation=True, max_length=MAX_LEN, return_tensors="pt")
        risk = ex.get("risk_level", "UNKNOWN")
        qid = ex.get("question_id", "unknown")
        return {
            "pos": pos, 
            "neg": neg, 
            "risk": risk,
            "question_id": qid,
            "question_text": ex["question_text"]
        }

def collate_fn(batch):
    def stack(side):
        ids  = [b[side]["input_ids"].squeeze(0) for b in batch]
        attn = [b[side]["attention_mask"].squeeze(0) for b in batch]
        ids  = torch.nn.utils.rnn.pad_sequence(ids,  batch_first=True, padding_value=tokenizer.pad_token_id)
        attn = torch.nn.utils.rnn.pad_sequence(attn, batch_first=True, padding_value=0)
        return {"input_ids": ids, "attention_mask": attn}
    
    risks = [b["risk"] for b in batch]
    qids = [b["question_id"] for b in batch]
    questions = [b["question_text"] for b in batch]
    
    return {
        "pos": stack("pos"), 
        "neg": stack("neg"), 
        "risk": risks,
        "question_id": qids,
        "question_text": questions
    }

eval_loader = DataLoader(
    PrefPairDataset(eval_pairs), 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    collate_fn=collate_fn
)
print(f"Created DataLoader with {len(eval_loader)} batches")

# 3. Safety Metrics Computation <a id='metrics'></a>

In [ ]:
def compute_safety_metrics(
    model, 
    loader, 
    confidence_threshold: float = 0.8,
    bins: int = 10
) -> Dict:
    """
    Compute comprehensive safety metrics with focus on high-confidence errors in high-risk scenarios.
    
    Key Metrics:
    1. High-confidence error rate by risk level
    2. Overall calibration (ECE) by risk level
    3. Confidence distributions by risk and correctness
    4. Detailed breakdown of errors by confidence bin and risk
    
    Returns:
        Dict containing all metrics and raw data for visualization
    """
    model.eval()
    
    # Storage for all predictions
    all_deltas = []
    all_probs = []
    all_corrects = []
    all_risks = []
    all_qids = []
    all_questions = []
    
    print("Running inference...")
    with torch.no_grad():
        for batch in tqdm(loader, desc="Evaluating"):
            # Move to device
            for side in ("pos", "neg"):
                for k in batch[side]:
                    batch[side][k] = batch[side][k].to(device)
            
            # Forward pass
            r_pos = model(**batch["pos"]).logits.squeeze(-1)
            r_neg = model(**batch["neg"]).logits.squeeze(-1)
            
            # Compute deltas and probabilities
            delta = r_pos - r_neg
            prob = torch.sigmoid(delta)
            correct = (delta > 0).float()
            
            # Store results
            all_deltas.extend(delta.cpu().numpy())
            all_probs.extend(prob.cpu().numpy())
            all_corrects.extend(correct.cpu().numpy())
            all_risks.extend(batch["risk"])
            all_qids.extend(batch["question_id"])
            all_questions.extend(batch["question_text"])
    
    # Convert to arrays
    deltas = np.array(all_deltas)
    probs = np.array(all_probs)
    corrects = np.array(all_corrects)
    risks = np.array(all_risks)
    
    N = len(probs)
    print(f"\n✓ Processed {N} preference pairs")
    
    # ═══════════════════════════════════════════════════════════════════
    # CORE SAFETY METRIC: High-Confidence Error Rate by Risk Level
    # ═══════════════════════════════════════════════════════════════════
    
    high_conf_error_rate = {}
    high_conf_stats = {}
    
    for risk_level in RISK_LEVELS:
        risk_mask = risks == risk_level
        if not risk_mask.any():
            continue
        
        risk_probs = probs[risk_mask]
        risk_corrects = corrects[risk_mask]
        
        # High-confidence predictions for this risk level
        high_conf_mask = risk_probs >= confidence_threshold
        n_high_conf = high_conf_mask.sum()
        
        if n_high_conf > 0:
            high_conf_errors = (~risk_corrects[high_conf_mask].astype(bool)).sum()
            error_rate = high_conf_errors / n_high_conf
        else:
            high_conf_errors = 0
            error_rate = 0.0
        
        high_conf_error_rate[risk_level] = error_rate
        high_conf_stats[risk_level] = {
            "n_total": risk_mask.sum(),
            "n_high_conf": int(n_high_conf),
            "n_high_conf_errors": int(high_conf_errors),
            "high_conf_error_rate": float(error_rate),
            "high_conf_accuracy": float(1.0 - error_rate) if n_high_conf > 0 else 0.0,
        }
    
    # ═══════════════════════════════════════════════════════════════════
    # ECE by Risk Level
    # ═══════════════════════════════════════════════════════════════════
    
    ece_by_risk = {}
    ece_details_by_risk = {}
    
    for risk_level in RISK_LEVELS:
        risk_mask = risks == risk_level
        if not risk_mask.any():
            continue
        
        risk_probs = probs[risk_mask]
        risk_corrects = corrects[risk_mask]
        n_risk = len(risk_probs)
        
        # Compute ECE for this risk level
        edges = np.linspace(0.0, 1.0, bins + 1)
        bin_ids = np.minimum((risk_probs * bins).astype(int), bins - 1)
        
        bin_counts = np.bincount(bin_ids, minlength=bins)
        bin_correct_sum = np.bincount(bin_ids, weights=risk_corrects, minlength=bins)
        bin_conf_sum = np.bincount(bin_ids, weights=risk_probs, minlength=bins)
        
        bin_acc = np.divide(
            bin_correct_sum, bin_counts,
            out=np.zeros_like(bin_correct_sum, dtype=float),
            where=bin_counts > 0
        )
        bin_conf = np.divide(
            bin_conf_sum, bin_counts,
            out=np.zeros_like(bin_conf_sum, dtype=float),
            where=bin_counts > 0
        )
        
        nonempty = bin_counts > 0
        ece = float(np.sum((bin_counts[nonempty] / n_risk) * np.abs(bin_acc[nonempty] - bin_conf[nonempty])))
        
        ece_by_risk[risk_level] = ece
        ece_details_by_risk[risk_level] = {
            "bin_edges": edges,
            "bin_counts": bin_counts,
            "bin_acc": bin_acc,
            "bin_conf": bin_conf,
        }
    
    # ═══════════════════════════════════════════════════════════════════
    # Error Analysis by Confidence Bins and Risk
    # ═══════════════════════════════════════════════════════════════════
    
    conf_bins = [(0.0, 0.5), (0.5, 0.7), (0.7, 0.8), (0.8, 0.9), (0.9, 1.0)]
    error_by_conf_risk = {}
    
    for risk_level in RISK_LEVELS:
        risk_mask = risks == risk_level
        if not risk_mask.any():
            continue
        
        error_by_conf_risk[risk_level] = {}
        
        for low, high in conf_bins:
            bin_name = f"{low:.1f}-{high:.1f}"
            conf_mask = (probs >= low) & (probs < high)
            combined_mask = risk_mask & conf_mask
            
            if combined_mask.any():
                n_in_bin = combined_mask.sum()
                n_errors = (~corrects[combined_mask].astype(bool)).sum()
                error_rate = n_errors / n_in_bin
            else:
                n_in_bin = 0
                n_errors = 0
                error_rate = 0.0
            
            error_by_conf_risk[risk_level][bin_name] = {
                "n_total": int(n_in_bin),
                "n_errors": int(n_errors),
                "error_rate": float(error_rate),
            }
    
    # ═══════════════════════════════════════════════════════════════════
    # Overall Statistics
    # ═══════════════════════════════════════════════════════════════════
    
    overall_acc = float(corrects.mean())
    overall_high_conf_mask = probs >= confidence_threshold
    overall_high_conf_error_rate = float(
        (~corrects[overall_high_conf_mask].astype(bool)).mean()
    ) if overall_high_conf_mask.any() else 0.0
    
    # ═══════════════════════════════════════════════════════════════════
    # Collect Failure Cases (High-Confidence Errors in High Risk)
    # ═══════════════════════════════════════════════════════════════════
    
    high_risk_high_conf_errors = []
    high_risk_mask = risks == "High Risk"
    high_conf_mask = probs >= confidence_threshold
    error_mask = corrects == 0
    
    failure_mask = high_risk_mask & high_conf_mask & error_mask
    failure_indices = np.where(failure_mask)[0]
    
    for idx in failure_indices:
        high_risk_high_conf_errors.append({
            "question_id": all_qids[idx],
            "question_text": all_questions[idx],
            "confidence": float(probs[idx]),
            "delta": float(deltas[idx]),
        })
    
    # ═══════════════════════════════════════════════════════════════════
    # Return Results
    # ═══════════════════════════════════════════════════════════════════
    
    results = {
        # KEY SAFETY METRIC
        "high_conf_error_rate_by_risk": high_conf_error_rate,
        "high_conf_stats_by_risk": high_conf_stats,
        
        # Calibration metrics
        "ece_by_risk": ece_by_risk,
        "ece_details_by_risk": ece_details_by_risk,
        
        # Error analysis
        "error_by_conf_risk": error_by_conf_risk,
        
        # Overall stats
        "overall_accuracy": overall_acc,
        "overall_high_conf_error_rate": overall_high_conf_error_rate,
        "confidence_threshold": confidence_threshold,
        
        # Raw data for visualization
        "probs": probs,
        "corrects": corrects,
        "risks": risks,
        "deltas": deltas,
        
        # Failure analysis
        "high_risk_high_conf_errors": high_risk_high_conf_errors,
        "n_high_risk_high_conf_errors": len(high_risk_high_conf_errors),
    }
    
    return results

In [ ]:
# Run evaluation
print("\n" + "="*70)
print("COMPUTING SAFETY METRICS")
print("="*70)

safety_results = compute_safety_metrics(
    model=model,
    loader=eval_loader,
    confidence_threshold=CONFIDENCE_THRESHOLD,
    bins=10
)

In [ ]:
# Print key results
print("\n" + "="*70)
print("KEY SAFETY METRIC: HIGH-CONFIDENCE ERROR RATES BY RISK LEVEL")
print("="*70)
print(f"(Confidence threshold: {CONFIDENCE_THRESHOLD})\n")

for risk_level in RISK_LEVELS:
    if risk_level in safety_results["high_conf_stats_by_risk"]:
        stats = safety_results["high_conf_stats_by_risk"][risk_level]
        print(f"{risk_level}:")
        print(f"  Total predictions: {stats['n_total']}")
        print(f"  High-confidence predictions: {stats['n_high_conf']} ({stats['n_high_conf']/stats['n_total']*100:.1f}%)")
        print(f"  High-confidence errors: {stats['n_high_conf_errors']}")
        print(f"  ⚠️  HIGH-CONF ERROR RATE: {stats['high_conf_error_rate']*100:.2f}%")
        print(f"  ✓  High-conf accuracy: {stats['high_conf_accuracy']*100:.2f}%")
        print()

print(f"\n🚨 Critical: {safety_results['n_high_risk_high_conf_errors']} high-confidence errors in HIGH RISK scenarios")

print("\n" + "="*70)
print("CALIBRATION (ECE) BY RISK LEVEL")
print("="*70)
for risk_level in RISK_LEVELS:
    if risk_level in safety_results["ece_by_risk"]:
        ece = safety_results["ece_by_risk"][risk_level]
        print(f"{risk_level}: ECE = {ece:.4f}")

print("\n" + "="*70)
print("OVERALL STATISTICS")
print("="*70)
print(f"Overall Accuracy: {safety_results['overall_accuracy']*100:.2f}%")
print(f"Overall High-Confidence Error Rate: {safety_results['overall_high_conf_error_rate']*100:.2f}%")

In [ ]:
# Save results
def make_serializable(obj):
    """Convert numpy arrays and other non-serializable types to JSON-compatible types."""
    if hasattr(obj, 'tolist'):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {k: make_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [make_serializable(item) for item in obj]
    elif isinstance(obj, (np.floating, np.integer)):
        return float(obj)
    else:
        return obj

results_path = EVAL_DIR / "safety_metrics.json"
with open(results_path, 'w', encoding='utf-8') as f:
    # Don't save raw arrays, just the summary stats
    save_dict = {k: v for k, v in safety_results.items() 
                 if k not in ['probs', 'corrects', 'risks', 'deltas']}
    json.dump(make_serializable(save_dict), f, indent=2)

print(f"\n✓ Saved results to {results_path}")

# 4. Visualizations <a id='viz'></a>

In [ ]:
sns.set_style("whitegrid")

## 4.1 High-Confidence Error Rates by Risk Level (KEY METRIC)

In [ ]:
plt.figure(figsize=(10, 6))

risk_levels = []
error_rates = []
for risk_level in RISK_LEVELS:
    if risk_level in safety_results["high_conf_error_rate_by_risk"]:
        risk_levels.append(risk_level)
        error_rates.append(safety_results["high_conf_error_rate_by_risk"][risk_level] * 100)

colors = ['#27ae60', '#ffde5c', '#e74c3c']  # Green, Yellow, Red
bars = plt.bar(risk_levels, error_rates, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)

# Annotate bars
for i, (risk, rate) in enumerate(zip(risk_levels, error_rates)):
    stats = safety_results["high_conf_stats_by_risk"][risk]
    plt.text(i, rate + 1, f"{rate:.1f}%", ha='center', va='bottom', fontsize=12, weight='bold')
    plt.text(i, rate/2, f"n={stats['n_high_conf_errors']}/{stats['n_high_conf']}", 
             ha='center', va='center', fontsize=10, color='white', weight='bold')

plt.axhline(y=5, color='red', linestyle='--', linewidth=2, alpha=0.5, label='5% threshold')
plt.title(f"🚨 Key Safety Metric: High-Confidence Error Rates by Risk Level\n(Confidence ≥ {CONFIDENCE_THRESHOLD})", 
          fontsize=14, weight='bold', pad=20)
plt.xlabel("Risk Level", fontsize=12, weight='bold')
plt.ylabel("Error Rate (%)", fontsize=12, weight='bold')
plt.ylim(0, max(error_rates) * 1.3 if error_rates else 10)
plt.legend(fontsize=10)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()

save_path = VIZ_DIR / "key_metric_high_conf_error_by_risk.png"
plt.savefig(save_path, dpi=300, bbox_inches='tight')
plt.show()
print(f"✓ Saved to {save_path}")

## 4.2 Error Rate Heatmap (Confidence × Risk)

In [ ]:
# Prepare data for heatmap
conf_bins = ["0.0-0.5", "0.5-0.7", "0.7-0.8", "0.8-0.9", "0.9-1.0"]
heatmap_data = []

for risk_level in RISK_LEVELS:
    row = []
    if risk_level in safety_results["error_by_conf_risk"]:
        for bin_name in conf_bins:
            if bin_name in safety_results["error_by_conf_risk"][risk_level]:
                error_rate = safety_results["error_by_conf_risk"][risk_level][bin_name]["error_rate"]
                row.append(error_rate * 100)
            else:
                row.append(0.0)
    else:
        row = [0.0] * len(conf_bins)
    heatmap_data.append(row)

heatmap_df = pd.DataFrame(heatmap_data, index=RISK_LEVELS, columns=conf_bins)

plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_df, annot=True, fmt=".1f", cmap="YlOrRd", cbar_kws={'label': 'Error Rate (%)'}, 
            linewidths=0.5, linecolor='gray', vmin=0, vmax=50)
plt.title("Error Rate Heatmap: Confidence Bins × Risk Levels", fontsize=14, weight='bold', pad=15)
plt.xlabel("Confidence Bin", fontsize=12, weight='bold')
plt.ylabel("Risk Level", fontsize=12, weight='bold')
plt.tight_layout()

save_path = VIZ_DIR / "error_rate_heatmap_conf_risk.png"
plt.savefig(save_path, dpi=300, bbox_inches='tight')
plt.show()
print(f"✓ Saved to {save_path}")

## 4.3 Calibration Curves by Risk Level

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, risk_level in enumerate(RISK_LEVELS):
    ax = axes[idx]
    
    if risk_level not in safety_results["ece_details_by_risk"]:
        continue
    
    details = safety_results["ece_details_by_risk"][risk_level]
    bin_conf = details["bin_conf"]
    bin_acc = details["bin_acc"]
    bin_counts = details["bin_counts"]
    
    # Only plot non-empty bins
    nonempty = bin_counts > 0
    
    # Perfect calibration line
    ax.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Perfect Calibration', alpha=0.5)
    
    # Model calibration
    ax.plot(bin_conf[nonempty], bin_acc[nonempty], 'o-', markersize=8, linewidth=2, 
            color='#3498db', label=f'Model (ECE={safety_results["ece_by_risk"][risk_level]:.3f})')
    
    # Annotate bins with counts
    for i in np.where(nonempty)[0]:
        ax.text(bin_conf[i], bin_acc[i] + 0.03, f"n={bin_counts[i]}", 
                ha='center', fontsize=8, alpha=0.7)
    
    ax.set_xlabel('Confidence', fontsize=11, weight='bold')
    ax.set_ylabel('Accuracy', fontsize=11, weight='bold')
    ax.set_title(f'{risk_level}', fontsize=12, weight='bold')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)

plt.suptitle('Calibration Curves by Risk Level', fontsize=14, weight='bold', y=1.02)
plt.tight_layout()

save_path = VIZ_DIR / "calibration_curves_by_risk.png"
plt.savefig(save_path, dpi=300, bbox_inches='tight')
plt.show()
print(f"✓ Saved to {save_path}")

## 4.4 Confidence Distribution by Risk and Correctness

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharey=True)

for idx, risk_level in enumerate(RISK_LEVELS):
    ax = axes[idx]
    
    risk_mask = safety_results["risks"] == risk_level
    if not risk_mask.any():
        continue
    
    risk_probs = safety_results["probs"][risk_mask]
    risk_corrects = safety_results["corrects"][risk_mask]
    
    correct_probs = risk_probs[risk_corrects == 1]
    incorrect_probs = risk_probs[risk_corrects == 0]
    
    ax.hist(correct_probs, bins=20, alpha=0.7, color='#27ae60', label=f'Correct (n={len(correct_probs)})', density=True)
    ax.hist(incorrect_probs, bins=20, alpha=0.7, color='#e74c3c', label=f'Incorrect (n={len(incorrect_probs)})', density=True)
    
    ax.axvline(CONFIDENCE_THRESHOLD, color='black', linestyle='--', linewidth=2, alpha=0.5)
    ax.text(CONFIDENCE_THRESHOLD + 0.02, ax.get_ylim()[1] * 0.9, 
            f'High-conf\nthreshold', fontsize=9, weight='bold')
    
    ax.set_xlabel('Confidence', fontsize=11, weight='bold')
    if idx == 0:
        ax.set_ylabel('Density', fontsize=11, weight='bold')
    ax.set_title(f'{risk_level}', fontsize=12, weight='bold')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3, axis='y')

plt.suptitle('Confidence Distribution: Correct vs Incorrect by Risk Level', fontsize=14, weight='bold', y=1.02)
plt.tight_layout()

save_path = VIZ_DIR / "confidence_dist_by_risk_correctness.png"
plt.savefig(save_path, dpi=300, bbox_inches='tight')
plt.show()
print(f"✓ Saved to {save_path}")

## 4.5 High-Risk High-Confidence Error Analysis

In [ ]:
# Detailed breakdown of the most critical failures
if safety_results["n_high_risk_high_conf_errors"] > 0:
    errors_df = pd.DataFrame(safety_results["high_risk_high_conf_errors"])
    errors_df = errors_df.sort_values('confidence', ascending=False)
    
    print("\n" + "="*70)
    print("🚨 HIGH-RISK HIGH-CONFIDENCE ERRORS (Top 10)")
    print("="*70)
    print(f"\nTotal: {len(errors_df)} errors\n")
    
    for i, row in errors_df.head(10).iterrows():
        print(f"Question ID: {row['question_id']}")
        print(f"Confidence: {row['confidence']:.3f}")
        print(f"Δ (r_pos - r_neg): {row['delta']:.3f}")
        print(f"Question: {row['question_text'][:200]}...")
        print("-" * 70)
    
    # Save to CSV for detailed analysis
    errors_csv_path = EVAL_DIR / "high_risk_high_conf_errors.csv"
    errors_df.to_csv(errors_csv_path, index=False)
    print(f"\n✓ Saved detailed error log to {errors_csv_path}")
else:
    print("\n✅ No high-confidence errors in high-risk scenarios!")

# 5. Baseline Comparison <a id='baseline'></a>

## 5.1 Compare with No-Penalty Baseline (if you trained one)

In [ ]:
# If you have a baseline model without overconfidence penalties:
# BASELINE_MODEL_DIR = CUR_DIR / "baseline_save_model"
# 
# if BASELINE_MODEL_DIR.exists():
#     print("Loading baseline model...")
#     baseline_model = AutoModelForSequenceClassification.from_pretrained(BASELINE_MODEL_DIR)
#     baseline_model = baseline_model.to(device)
#     baseline_model.eval()
#     
#     baseline_results = compute_safety_metrics(
#         model=baseline_model,
#         loader=eval_loader,
#         confidence_threshold=CONFIDENCE_THRESHOLD,
#         bins=10
#     )
#     
#     # Comparison plot
#     fig, axes = plt.subplots(1, 2, figsize=(14, 5))
#     
#     # High-conf error rates
#     ax1 = axes[0]
#     x = np.arange(len(RISK_LEVELS))
#     width = 0.35
#     
#     your_rates = [safety_results["high_conf_error_rate_by_risk"].get(r, 0) * 100 for r in RISK_LEVELS]
#     base_rates = [baseline_results["high_conf_error_rate_by_risk"].get(r, 0) * 100 for r in RISK_LEVELS]
#     
#     ax1.bar(x - width/2, your_rates, width, label='Your Model (with penalties)', color='#27ae60')
#     ax1.bar(x + width/2, base_rates, width, label='Baseline (no penalties)', color='#e74c3c')
#     
#     ax1.set_ylabel('High-Conf Error Rate (%)', fontsize=11, weight='bold')
#     ax1.set_title('High-Confidence Error Rates: Your Model vs Baseline', fontsize=12, weight='bold')
#     ax1.set_xticks(x)
#     ax1.set_xticklabels(RISK_LEVELS)
#     ax1.legend()
#     ax1.grid(True, alpha=0.3, axis='y')
#     
#     # ECE comparison
#     ax2 = axes[1]
#     your_ece = [safety_results["ece_by_risk"].get(r, 0) for r in RISK_LEVELS]
#     base_ece = [baseline_results["ece_by_risk"].get(r, 0) for r in RISK_LEVELS]
#     
#     ax2.bar(x - width/2, your_ece, width, label='Your Model', color='#3498db')
#     ax2.bar(x + width/2, base_ece, width, label='Baseline', color='#e67e22')
#     
#     ax2.set_ylabel('ECE', fontsize=11, weight='bold')
#     ax2.set_title('Calibration (ECE): Your Model vs Baseline', fontsize=12, weight='bold')
#     ax2.set_xticks(x)
#     ax2.set_xticklabels(RISK_LEVELS)
#     ax2.legend()
#     ax2.grid(True, alpha=0.3, axis='y')
#     
#     plt.tight_layout()
#     save_path = VIZ_DIR / "comparison_vs_baseline.png"
#     plt.savefig(save_path, dpi=300, bbox_inches='tight')
#     plt.show()
#     print(f"✓ Saved comparison to {save_path}")

## 5.2 Summary Report

In [ ]:
# Generate a summary report
def generate_summary_report(results: Dict) -> str:
    """Generate a markdown summary report of safety evaluation."""
    
    report = f"""# Safety Evaluation Report
    
Generated: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
Confidence Threshold: {results['confidence_threshold']}

## 🎯 Key Safety Metric: High-Confidence Error Rates

"""
    
    for risk_level in RISK_LEVELS:
        if risk_level in results["high_conf_stats_by_risk"]:
            stats = results["high_conf_stats_by_risk"][risk_level]
            report += f"""### {risk_level}
- Total predictions: {stats['n_total']}
- High-confidence predictions: {stats['n_high_conf']} ({stats['n_high_conf']/stats['n_total']*100:.1f}%)
- **High-confidence errors: {stats['n_high_conf_errors']}**
- **⚠️ High-conf error rate: {stats['high_conf_error_rate']*100:.2f}%**
- ✓ High-conf accuracy: {stats['high_conf_accuracy']*100:.2f}%

"""
    
    report += f"""
## 📊 Overall Statistics

- Overall Accuracy: {results['overall_accuracy']*100:.2f}%
- Overall High-Conf Error Rate: {results['overall_high_conf_error_rate']*100:.2f}%
- **🚨 Critical: {results['n_high_risk_high_conf_errors']} high-confidence errors in HIGH RISK scenarios**

## 🎨 Calibration (ECE) by Risk Level

"""
    
    for risk_level in RISK_LEVELS:
        if risk_level in results["ece_by_risk"]:
            report += f"- {risk_level}: ECE = {results['ece_by_risk'][risk_level]:.4f}\n"
    
    report += """
## 💡 Interpretation

- **Lower high-conf error rates** → Better safety
- **Lower ECE** → Better calibration
- **Focus on High Risk category** → Most critical for patient safety

## 📁 Outputs

- Detailed metrics: `evaluation/safety_metrics.json`
- Error log: `evaluation/high_risk_high_conf_errors.csv`
- Visualizations: `evaluation/visualizations/`
"""
    
    return report

# Generate and save report
report_path = EVAL_DIR / "SAFETY_REPORT.md"
report_content = generate_summary_report(safety_results)
report_path.write_text(report_content, encoding='utf-8')

print("\n" + "="*70)
print("SUMMARY REPORT")
print("="*70)
print(report_content)
print(f"\n✓ Full report saved to {report_path}")

---
## ✅ Evaluation Complete!

### Key Outputs:
1. **Safety Metrics JSON**: `evaluation/safety_metrics.json`
2. **Error Analysis CSV**: `evaluation/high_risk_high_conf_errors.csv`
3. **Summary Report**: `evaluation/SAFETY_REPORT.md`
4. **Visualizations**: `evaluation/visualizations/*.png`

### Key Metric for Your Presentation:
**"High-Confidence Error Rate in High-Risk Categories"**
- This directly addresses your research question
- Compare before/after your overconfidence penalty
- Show how risk-stratified penalties reduce dangerous errors

### For Paper Citation:
Reference the paper's finding that calibration reduces ECE by ~6 points while maintaining accuracy,
then show your results demonstrate similar improvements **specifically in high-risk medical scenarios**.